https://adventofcode.com/2022/day/7

In [1]:
f = open("input.txt", "r")

data = []

while ! eof(f)
	s = readline(f)
	
	# push each line in data array
	push!(data, s)
end

close(f)

In [2]:
# make an object class (struct) that contains the properties "name", "size", "children", and "parent"
# note that children and parent are members of the struct
mutable struct Directory
	name::String
	size::Int
	children::Array{Directory}
	parent::Union{Directory,Nothing}
end

# creates a data structure mirroring the file system
function generateDirectories(data)
	# array of all directories
	directories = Directory[]

	# creates a root directory to start with
	parent = nothing
	root = Directory("root", 0, Directory[], nothing)
	current = root

	# loops through each log in the data
	for log in data
		# if the log contains "$ cd" we are changing our directory
		if match(r"\$ cd", log) != nothing

			# remove the "$ cd " returning the name of the directory
			name = replace(log,"\$ cd "=>"")

			# if name ".." we are moving out the directory to the parent
			if name == ".."
				current = parent
				parent = current.parent
			else
				# if name isn't ".." we are in a new directory and create one accordingly
				parent = current
				current = Directory(name, 0, Directory[], parent)

				# update the parent's children array with this child
				push!(parent.children, current)
				# place the new directory in the full list of directories
				push!(directories, current)
			end
		end

		# if the log has numbers it's a file
		if match(r"\d+", log) != nothing
			# use regex to parse the size number of the file
			file_size = parse(Int,match(r"\d+", log).match)

			# add the size to the current directory
			current.size += file_size

			# get the parent of the current directory
			super_directory = current.parent

			# add the file size to all the parent directories until the root
			while super_directory.parent != nothing
				super_directory.size += file_size
				super_directory = super_directory.parent
			end
		end
	end

	return directories
end

# gets the sum of all directories over 100000
function sum100000(data)
	directories = generateDirectories(data)
	sum = 0
	for directory in directories
		size = directory.size
		if size <= 100000
			sum += size
		end
	end

	return sum
end

sum100000(data)

1182909

In [3]:
# finds the size of the directory needing to be deleted
function findDeleteDirectorySize(data)
	total_space = 70000000
	update_size = 30000000
	# creates the directories
	directories = generateDirectories(data)

	# sorts the directories by size smallest -> biggest
	sorted_directories = sort!(directories, by = directory -> directory.size)

	# calculates free space of device (the last directory is the "/" which has total use space)
	free_space = total_space - sorted_directories[end].size

	# calculates the minimum size of directory to be deleted
	min_size_directory = update_size - free_space

	# loops and find the first directory that is greater than the minimum size
	for directory in sorted_directories
		if directory.size > min_size_directory
			return directory.size
		end
	end
end

findDeleteDirectorySize(data)

2832508